### Import

In [ ]:
from langchain_google_genai import (
    GoogleGenerativeAI, 
    ChatGoogleGenerativeAI,
)
from langchain_core.messages import HumanMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import (
    BaseMessage,
    SystemMessage,
    HumanMessage,
    AIMessage,
)
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField


### Environment

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")


### Models

In [ ]:
llm = GoogleGenerativeAI(
    model="gemini-2.0-flash",
    max_tokens=50
)
print(llm.__class__)

# specify "max_output_tokens" as configurable fields
llm2 = llm.configurable_fields(
    max_output_tokens=ConfigurableField(
        id="max_output_tokens",
        name="LLM Maximum output tokens",
        description="Maximum number of tokens",
    )
)

In [ ]:
chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    max_tokens=50
)
print(chat.__class__)

# specify "max_output_tokens" and "temperature" 
# as configurable fields
chat2 = chat.configurable_fields(
    max_output_tokens=ConfigurableField(
        id="max_output_tokens",
        name="LLM Maximum output tokens",
        description="Maximum number of tokens",
    ),
    temperature=ConfigurableField(
        id="temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

### Example - LLM

In [ ]:
response = llm2.invoke(
        "What are some of the pros and cons of Python as a programming language?",
        # specify optional configurable field
        config={"configurable": {"max_output_tokens": 500}} 
    )
print(response.__class__)
print(response)

In [ ]:
prompt_template = PromptTemplate.from_template(
    """Extract the following entities:\n{entities}\n 
from the item's description:\n{description}.\n
You always respond in a json format."""
)

chain = prompt_template | llm | JsonOutputParser()
print(chain.__class__)

In [ ]:
result = chain.invoke({"entities":"price, RAM", "description": "Price: $1,000 and RAM: 32GB"})
print(result.__class__)
print(result)

### Example - Chat

In [ ]:
response = chat2.invoke("Sing a ballad of LangChain.", 
    # specify optional configurable field
    config={"configurable": {"max_output_tokens": 300}})
print(response.__class__)
print(response.usage_metadata)
print(response.content)

In [ ]:
for chunk in chat.stream("Sing a ballad of LangChain."):
    print(chunk.content, end="")

In [ ]:
for chunk in (chat | StrOutputParser()).stream("Sing a ballad of LangChain."):
    print(chunk, end="")

In [ ]:
response = (chat | StrOutputParser()).invoke("Sing a ballad of LangChain.")
print(response.__class__)
print(response)

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """You are a helpful assistant that helps extract entities from 
product descriptions. You always respond in a json format.""")
        ),
        HumanMessagePromptTemplate.from_template(
            """Extract the following entities:\n{entities}\n 
from the item's description:\n{description}."""),
    ]
)

chain = prompt_template | chat2 | JsonOutputParser()
print(chain.__class__)

In [ ]:
result = chain.invoke({"entities":"price, RAM", "description": "Price: $1,000 and RAM: 32GB"},
    # specify optional configurable field
    config={"configurable": {"temperature": 0.1}})
print(result.__class__)
print(result)

### Example - Image Analysis

In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)

response = chat.invoke([message])
print(response.content)